In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
import random
from random import *
import pandas
import numpy as np
import os
import sys
import PIL
from PIL import Image


In [2]:
root='/home/zeno/pytorchnotebooks/dataset/'

dogstestset=''.join(root+'test_set/'+'dogs/')
catstestset=''.join(root+'test_set/'+'cats/')
dogstrainset=''.join(root+'training_set/'+'dogs/')
catstrainset=''.join(root+'training_set/'+'cats/')

print(len(os.listdir(dogstrainset)))
print(len(os.listdir(catstrainset)))
print(len(os.listdir(dogstestset)))
print(len(os.listdir(catstestset)))

4001
4001
1001
1001


In [6]:
#use when want to show randomly selected sample image
sample_index=randint(1,4001)
dog_img=os.listdir(dogstrainset)[sample_index]
path2dog_img=dogstrainset+dog_img
cat_img=os.listdir(catstrainset)[sample_index]
path2cat_img=catstrainset+cat_img
from PIL import Image

%matplotlib inline
imd = Image.open(path2dog_img).convert('L') # use only gray 
imc=Image.open(path2cat_img).convert('L')

resized_d.show()
resized_c.show()
#resize the image to 128 128
resized_d=imd.resize((128,128))
resized_c=imc.resize((128,128))

resized_d.show()
resized_c.show()
# convert to np array
dog_np=np.asarray(resized_d)
cat_np=np.asarray(resized_c)

print(dog_np.shape,cat_np.shape)


(128, 128) (128, 128)


In [3]:
# convert all image to equal size and put into np array and add labels
cat_or_dog=["cat","dog"]
def image_to_numpy(path,cat_or_dog):
    #  cat is 0. and dog is 1.
    ls=[]
    l=len(os.listdir(path))-1 # last one is .DS_Store 
    
    for file in os.listdir(path):
        if file.endswith('.jpg'):
            #print('process file:{}'.format(file))
            im=Image.open(path+file).convert('L')
            im_resized=im.resize((80,80))
            im_to_np=np.asarray(im_resized,dtype=np.float32)
            ls.append(im_to_np)
            im.close()
        else:
            pass
    return [cat_or_dog for i in range(len(ls))], ls
    
test_label_dog,test_dog=image_to_numpy(dogstestset,1.)
test_label_cat,test_cat=image_to_numpy(catstestset,0.)
train_label_dog,train_dog=image_to_numpy(dogstrainset,1.)
train_label_cat,train_cat=image_to_numpy(catstrainset,0.)
print("num of imgs:{}, shape(img):{}, label:{}".format(len(test_dog),test_dog[0].shape,test_label_dog[0]))
print("num of imgs:{}, shape(img):{}, label:{}".format(len(test_cat),test_cat[0].shape,test_label_cat[0]))
print("num of imgs:{}, shape(img):{}, label:{}".format(len(train_dog),train_dog[0].shape,train_label_dog[0]))
print("num of imgs:{}, shape(img):{}, label:{}".format(len(train_cat),train_cat[0].shape,train_label_cat[0]))


num of imgs:1000, shape(img):(80, 80), label:1.0
num of imgs:1000, shape(img):(80, 80), label:0.0
num of imgs:4000, shape(img):(80, 80), label:1.0
num of imgs:4000, shape(img):(80, 80), label:0.0


In [4]:
#compose train_dat, train_label as well as test_data ,test_label are all numpy arrays now

import itertools
train_dat=list(itertools.chain(train_dog,train_cat)) 
train_dat=np.array(train_dat).reshape(8000,1,80,80)

train_label=list(itertools.chain(train_label_dog,train_label_cat))
train_label=np.array([[item] for item in train_label])

test_dat=list(itertools.chain(test_dog,test_cat))
test_dat=np.array(test_dat).reshape(2000,1,80,80)

test_label=list(itertools.chain(test_label_dog,test_label_cat))
test_label=np.array([[item] for item in test_label])


print(len(train_dat),train_dat[0].shape,train_dat[4001].shape)
print(len(train_label), train_label[0].shape, train_label[4001].shape)
print(len(test_dat),test_dat[0].shape)
print(len(test_label))
#test_dat,test_label

8000 (1, 80, 80) (1, 80, 80)
8000 (1,) (1,)
2000 (1, 80, 80)
2000


In [5]:
from random import  *
class SampleMaker(object):
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
    
    def push(self, event):
        #each event is a tuple of numpy arrays label 1,1 and dat(128,128)
        #convert to torch.floatTensor
        event2torch=event[0]       
        event_label=event[1]       
        self.memory.append([event2torch,event_label])  

  
        if len(self.memory) > self.capacity:
            del self.memory[0]
    
    def make(self, batch_size):
        samples = sample(*(self.memory, batch_size))
       
        return samples
        #return map(lambda x: Variable(torch.cat(x, 0)), samples)

In [6]:
#populate the SampleMaker 
samplemaker=SampleMaker(capacity=8000)
for i in range(8000):
    #print(test_dat[i].shape, test_label[i].shape)
    samplemaker.push([train_dat[i],train_label[i]])    
    

In [7]:
import itertools
def type_converter(sam,batch_size):
    train_placeholder=np.empty((1,80,80),dtype=float,order='C')
    train_labels=np.empty((1,1),dtype=float,order='C')
    batch=map(lambda s: s[0],sam)
    #convert batch
    for o in enumerate(batch):
        #print(o[1].shape)
        train_placeholder=np.vstack((train_placeholder,o[1]))
    train_batches=np.delete(train_placeholder, 0, 0).reshape(batch_size,1,80,80) #delete the empty placeholder
    
    batch_labels=list(map(lambda s:s[1],sam))
    batch_labels=[l.tolist() for l in batch_labels]#convert to list
    batch_labels=list(itertools.chain(*batch_labels))#flattern to flatlist
    batch_labels=[int(i) for i in batch_labels]#turn each inner-list type to int 
    batch_labels=np.array(batch_labels)#convert to np array
    #returning nd array for train batches and labels
    return train_batches,batch_labels

In [ ]:
"""# Samplemaker's output
samples=sample(*(samplemaker.memory,10))
t,l=type_converter(samples,10)
print(t.shape, type(t))
print(type(l), l)
"""

In [ ]:
"""
#convert the train batches and labels to torch tensors and put into the learn
tr_batches=torch.from_numpy(t).float()
tr_labels=torch.from_numpy(l)
print(type(tr_batches),tr_batches.size())
print(type(tr_labels),tr_labels.size())
"""

In [8]:
#covert the test data and labels into torch tensors
test_load=torch.from_numpy(test_dat).float()
print(type(test_load),test_load.size())
test_label=torch.from_numpy(test_label)
print(type(test_label),test_label.size())

<class 'torch.FloatTensor'> torch.Size([2000, 1, 80, 80])
<class 'torch.DoubleTensor'> torch.Size([2000, 1])


In [ ]:
"""for i in range(2000):
    print(test_dat[i].size(),test_label[i].size())"""

In [9]:
num_class=2


In [10]:
class CNN(nn.Module):
    
    def __init__(self, num_class):
        super(CNN, self).__init__()
        # in_channel=1 color , out_channel=hyper param, kernerl_size=hyper param
        self.convolution1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 5)
        # in_channel=32 previous out_channel , out_channel=hyper param, kernerl_size=hyper param
        self.convolution2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)
        # in_channel=32=previous out_channel , out_channel=hyper param, kernerl_size=hyper param
        self.convolution3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 2)
        #to obtain numOfNeurons to input to fc1, we use a function count_neurons()
        self.fc1 = nn.Linear(in_features = self.count_neurons((1,80,80)), out_features = 40)
        self.fc2 = nn.Linear(in_features = 40, out_features = num_class)

    def count_neurons(self, image_dim):
        #image_dim=(1,80,80) obtained from image_preprocessing.py
        x = Variable(torch.rand(1, *image_dim))#create fake image comply to dimensions feed to pytorch Variable
        #print('torch Variable x',x.size())   # variable 1x1x80x80
        x = F.relu(F.max_pool2d(self.convolution1(x), 3, 2))
        #print('first conv-maxpool x',x.size())# 1st conv-maxpool 1x32x37x37 torch.FloatTensor
        x = F.relu(F.max_pool2d(self.convolution2(x), 3, 2))
        #print('second conv-maxpool x',x.size())#2nd conv-maxpool 1x32x17x17 
        x = F.relu(F.max_pool2d(self.convolution3(x), 3, 2))
        #print('3rd conv-maxpool x',x.size())#3rd conv-maxpool 1x64x7x7
        #pytorch trick to get shape/num of neurons out of Conv  layers
        #print('x.data',x.data) #1x64x7x7
        #print('x.data.view(1,-1)',x.data.view(1,-1))
        #x.data.view(1,-1) is a torch.FloatTensor 1x3136 
        # 0.0000  0.0000  0.0000  ...   0.1507  0.1516  0.1373 
        return x.data.view(1, -1).size(1) # class :int

    def forward(self, x):
        #print(x.size())
        y = F.relu(F.max_pool2d(self.convolution1(x), 3, 2)) # apply max_pool2d here
        y = F.relu(F.max_pool2d(self.convolution2(y), 3, 2)) #apply max_pool2d here
        y = F.relu(F.max_pool2d(self.convolution3(y), 3, 2)) #apply max_pool2d here
        y = y.view(x.size(0), -1) # similar way to extract total num of neurons to feed fc1
        y = F.relu(self.fc1(y))
        y = self.fc2(y)
        return y
 

In [ ]:
#model=CNN(2)
#print(model.count_neurons((1,80,80)))



In [11]:
class dqn():
    def __init__(self, num_class):
        self.reward_window = []
        self.model = CNN(num_class)
        self.optimizer = optim.Adam(self.model.parameters(), lr = 0.001)
        self.criterion = nn.CrossEntropyLoss()

    def make_prediction(self, img):
        #img has to be a torch tensor of size 1,80,80
        #print(img.unsqueeze(0).size())
        test_outputs = self.model.forward(Variable(img.unsqueeze(0)))
        #print("test_outputs:{}".format(test_outputs))
                
        # Get predictions from the maximum value
        probs = F.softmax(test_outputs) # T=100
        print("probs:{}",probs)
        action = probs.multinomial()
        print(action)
        #print("predicted:{}".format(predicted))
        #predicted=predicted.double() # convert Longtensor to DoubleTensor
        #pred=predicted.numpy().tolist()[0]
        
        return action.data[0,0]     

        
    
    def learn(self, batch, targets, test,test_labels,load_model=False):
        images = Variable(batch)
        labels = Variable(targets)
        iter=0
        if load_model:
            self.load()
        
        # Clear gradients w.r.t. parameters
        self.optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = self.model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = self.criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        self.optimizer.step()             
        
        
        # Calculate Accuracy         
        correct = 0
        false_positive=0
        false_negative=0
        total = len(test_labels)
        # Iterate through test dataset
        
        test_imgs = Variable(test)
        #test_labels = Variable(test_labels)
        
        for i in range(len(test_imgs)):
        # Forward pass only to get logits/output
            test_outputs = self.model(test_imgs[i].unsqueeze(0))
                
            # Get predictions from the maximum value
            _, predicted = torch.max(test_outputs.data, 1)
            predicted=predicted.double() # convert Longtensor to DoubleTensor
            #print('i:{}, type of predicted:{},predicted'.format(i,type(predicted),predicted))
            # Total number of labels
            #print('i: {},type of test_labels[i]:{},test_labels[i];{}'.format(i,type(test_labels[i]),test_labels[i]))
            #convert torch tensors back to numpy array and then list and then extract emelemnt inside the list
            #print(predicted.numpy().tolist()[0],test_labels[i].numpy().tolist()[0])
            pred=predicted.numpy().tolist()[0] # now it is of type float
            label=test_labels[i].numpy().tolist()[0]# now it is of type float
            
                        
            if (pred ==label):
                correct += 1
            elif (pred==1.0 and label==0.):
                false_positive+=1
            else:
                false_negative+=1
                #print("predicted:{},test_labels[{}]:{}".format(predicted,i,test_labels[i]))
        accuracy = 100 * correct / total
        iter+=1
        # Print Loss
        print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))
    def save(self):
        torch.save({'state_dict': self.model.state_dict(),
                    'optimizer' : self.optimizer.state_dict(),
                   }, 'last_brain.pth')
    def load(self):
        if os.path.isfile('last_brain.pth'):
            print("=> loading checkpoint... ")
            checkpoint = torch.load('last_brain.pth')
            self.model.load_state_dict(checkpoint['state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer'])
            print("done !")
        else:
            print("no checkpoint found...")


In [12]:
seq_model=dqn(2)
#run learning and make prediction base on batch_size
num_batches=10
batch_size=500
for i in range(num_batches):
    samples=sample(*(samplemaker.memory,batch_size))
    t,l=type_converter(samples,batch_size)
    tr_batches=torch.from_numpy(t).float()
    tr_labels=torch.from_numpy(l)
    seq_model.learn(tr_batches, tr_labels,test_load,test_label,load_model=True)
   

=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6450989246368408. Accuracy: 65.85
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6083582043647766. Accuracy: 65.4
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6198220252990723. Accuracy: 65.0
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6228146553039551. Accuracy: 65.0
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6062139272689819. Accuracy: 64.3
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6217004656791687. Accuracy: 65.5
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6076483130455017. Accuracy: 64.45
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6200554370880127. Accuracy: 65.25
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6462082862854004. Accuracy: 65.15
=> loading checkpoint... 
done !
Iteration: 1. Loss: 0.6337891817092896. Accuracy: 65.2


In [ ]:
# one can create epoch like this 
num_epoch=10
for epoch in range(num_epoch):
    for i in range(num_batches):
    samples=sample(*(samplemaker.memory,batch_size))
    t,l=type_converter(samples,batch_size)
    tr_batches=torch.from_numpy(t).float()
    tr_labels=torch.from_numpy(l)
    seq_model.learn(tr_batches, tr_labels,test_load,test_label,load_model=True) # set load_model=true to load the saved model
    # in each epoch, remember to save the model
    seq_model.save()
    

In [13]:
#seq_model.save()

In [14]:
seq_model=dqn(2)
m=seq_model.load()

=> loading checkpoint... 
done !


In [15]:
#use when want to show randomly selected sample image

img1=os.listdir(root+'single_prediction/')[0]
img2=os.listdir(root+'single_prediction/')[1]

from PIL import Image

%matplotlib inline
im1 = Image.open(root+'single_prediction/'+img1).convert('L') # use only gray 
im2=Image.open(root+'single_prediction/'+img2).convert('L')

#resize the image to 80,80
resized_1=im1.resize((80,80))
resized_2=im2.resize((80,80))

# convert to np array
im1_np=np.asarray(resized_1).reshape(1,80,80)
#print(im1_np)
im2_np=np.asarray(resized_2).reshape(1,80,80)
im1_tr=torch.from_numpy(im1_np).float()
im2_tr=torch.from_numpy(im2_np).float()
#print(im1_tr.size(),im2_tr.size())

#make the prediction
out1=seq_model.make_prediction(im1_tr)
print(out1)
out2=seq_model.make_prediction(im2_tr)
print(out2)

probs:{} Variable containing:
 0.3666  0.6334
[torch.FloatTensor of size 1x2]

Variable containing:
 0
[torch.LongTensor of size 1x1]

0
probs:{} Variable containing:
 0.4514  0.5486
[torch.FloatTensor of size 1x2]

Variable containing:
 1
[torch.LongTensor of size 1x1]

1


In [17]:
cat_or_dog=["cat","dog"]
print("cat_or_dog_2.jpg is a cat, prediction is out2: {}".format(cat_or_dog[out2]) )
print("cat_or_dog_1.jpg is a dog, prediction is out1 :{}".format(cat_or_dog[out1]) )

cat_or_dog_2.jpg is a cat, prediction is out2: dog
cat_or_dog_1.jpg is a dog, prediction is out1 :cat
